# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [48]:
# import packages and modifying output settings
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from collections import OrderedDict
# Remember to enter in the cell twice for this to work

# Loading files
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Creating Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Merging datasets into one DataFrame
school_data_analysis = pd.merge(student_data, school_data, how="left", 
                                on=["school_name", "school_name"])

# Generating new columns of math_pass and reading_pass if original 
# scores are >= 70 here to call upon later on
school_data_analysis.loc[:,'math_pass'] = np.where(school_data_analysis['math_score'] >= 70, True, False)
school_data_analysis.loc[:,'reading_pass'] = np.where(school_data_analysis['reading_score'] >= 70, True, False)
school_data_analysis.head(5)

# Changelog: for posterity's sake: 
# - To make the dataframes, make columns; initially tried making lists into dataframes,
# and that got seriously messy. Had to just combine manually by 'name': variable, where
# the variables were storing lists with relevant information.
# - Remember that manipulating a groupby object with a method returns a series.
# Additionally, chaining methods is awesome, .tolist() is a friend.
# - Cannot run operations on lists, need to trsnform them to np.arrays and change them
# back to lists.
# - Removing columns and creating a new dataframe from that caused problems with 
# how I was initially generating new columns for math_pass and reading_pass, since
# apparently a copy() is required to make sure I didn't get SettingWithCopyWarning
# - Stylistic note for this assigment: _calc and _pre are my interim 
# step variables to make the variables I needed.

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,math_pass,reading_pass
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,True,False
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,False,True
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,False,True
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,False,False
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,True,True


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Total number of schools
school_total = school_data_analysis['school_name'].nunique()

In [3]:
# Total number of students
student_total = school_data_analysis['Student ID'].count()

In [4]:
# Total budget
district_total_budget = sum(school_data_analysis['budget'].unique())

In [5]:
# Average math score 
avg_math_score = round(school_data_analysis['math_score'].mean(),2)

In [6]:
# Average reading score
avg_read_score = round(school_data_analysis['reading_score'].mean(),2)

In [7]:
# Overall passing rate 
overall_pass_rate = round(((school_data_analysis['math_score'].mean()) + 
                           (school_data_analysis['reading_score'].mean())) / 2, 2)

In [8]:
# Percentage of students with a passing math score (70 or greater)
math_pass_calc = round(((school_data_analysis['math_score'] >= 70)
                        .value_counts("True")) * 100, 2).tolist()
math_pass = math_pass_calc[0]

In [9]:
# Percentage of students with a passing reading score (70 or greater)
reading_pass_calc = round(((school_data_analysis['reading_score'] >= 70)
                           .value_counts("True")) * 100, 2).tolist()
reading_pass = reading_pass_calc[0]

In [26]:
# Creating dataframe with the above datapoints

# school_data_analysis["district_total_budget"] = school_data_analysis["district_total_budget"].map("${:.2f}".format)

dataSummary_overview = pd.DataFrame({'School Total': school_total, 'Student Total': student_total,
                            'District Total Budget': district_total_budget,'Average Math Score':
                            avg_math_score,'Average Reading Score': avg_read_score,
                            'Overall Passing %': overall_pass_rate, 'Math Passing % (score >= 70)':
                            math_pass, 'Reading Passing % (score >= 70)': reading_pass},index=[0])

dataSummary_overview

,School Total,Student Total,District Total Budget,Average Math Score,Average Reading Score,Overall Passing %,Math Passing % (score >= 70),Reading Passing % (score >= 70)
0,15,39170,24649428,78.99,81.88,80.43,74.98,85.81


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [11]:
# Creating a groupby object by school_name and type
byschool_summary = school_data_analysis.groupby(['school_name','type'])

# Making a list of school names
school_names = byschool_summary.median().index.get_level_values(0).tolist()

# Making a list of school type
school_type = byschool_summary.median().index.get_level_values(1).tolist()

In [12]:
# Total number of students
byschool_student_total = round(school_data_analysis['school_name']
                               .value_counts(),2).tolist()

# Creating an array for further calculations
byschool_student_total_calc = np.asarray(byschool_student_total)

In [13]:
# School budgets
byschool_budgets_total = school_data_analysis.groupby(['school_name']).mean()['budget'].tolist()
    
# Creating an array for further calculations
byschool_budgets_total_calc = np.asarray(byschool_budgets_total)

In [14]:
# Budget amount spent per student
byschool_budget_per = (byschool_budgets_total_calc / byschool_student_total_calc).tolist()

In [15]:
# Average math scores by school
byschool_math_avgs = round(school_data_analysis.groupby(['school_name']).mean()
                           ['math_score'],2).tolist()

In [16]:
# Average reading scores by school
byschool_reading_avgs = round(school_data_analysis.groupby(['school_name']).mean()
                              ['reading_score'],2).tolist()

In [17]:
# Percentage of students passing math
byschool_math_pass = round((school_data_analysis.groupby(['school_name']).mean()
                            ['math_pass'])*100,2).tolist()

# Creating an array for further calculations
byschool_math_pass_calc = np.asarray(byschool_math_pass)

In [18]:
# Percentage of students passing reading
byschool_reading_pass = round((school_data_analysis.groupby(['school_name']).mean()
                               ['reading_pass'])*100,2).tolist()

# Creating an array for further calculations
byschool_reading_pass_calc = np.asarray(byschool_reading_pass)

In [19]:
# Overall passing percentage
byschool_overall_pass =((byschool_math_pass_calc + byschool_reading_pass_calc)/2).tolist()

In [27]:
# Creating dataframe with the above datapoints
dataSummary_byschool = pd.DataFrame({'School Name': school_names, 'Type': school_type, 
                             'Total Students': byschool_student_total, 'Total Budgets':
                            byschool_budgets_total, 'Per Student Budgets': byschool_budget_per,
                             'Average Math Score': byschool_math_avgs, 'Average Reading Score':
                             byschool_reading_avgs, '% Passing Math': byschool_math_pass,
                             '% Passing Reading': byschool_reading_pass,
                             '% Overall Passing': byschool_overall_pass
                            })
dataSummary_byschool              

,School Name,Type,Total Students,Total Budgets,Per Student Budgets,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,3124928.0,628.000000,77.05,81.03,66.68,81.93,74.305
1,Cabrera High School,Charter,4761,1081356.0,227.127914,83.06,83.98,94.13,97.04,95.585
2,Figueroa High School,District,4635,1884411.0,406.561165,76.71,81.16,65.99,80.74,73.365
3,Ford High School,District,3999,1763916.0,441.089272,77.10,80.75,68.31,79.30,73.805
4,Griffin High School,Charter,2949,917500.0,311.122414,83.35,83.82,93.39,97.14,95.265
5,Hernandez High School,District,2917,3022020.0,1036.002743,77.29,80.93,66.75,80.86,73.805
6,Holden High School,Charter,2739,248087.0,90.575758,83.80,83.81,92.51,96.25,94.380
7,Huang High School,District,2283,1910635.0,836.896627,76.63,81.18,65.68,81.32,73.500
8,Johnson High School,District,1858,3094650.0,1665.581270,77.07,80.97,66.06,81.22,73.640
9,Pena High School,Charter,1800,585858.0,325.476667,83.84,84.04,94.59,95.95,95.270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [28]:
Top5Schools = dataSummary_byschool.sort_values('% Overall Passing',axis=0,ascending=False)
Top5Schools.head(5)

,School Name,Type,Total Students,Total Budgets,Per Student Budgets,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
1,Cabrera High School,Charter,4761,1081356.0,227.127914,83.06,83.98,94.13,97.04,95.585
12,Thomas High School,Charter,1468,1043130.0,710.579019,83.42,83.85,93.27,97.31,95.290
9,Pena High School,Charter,1800,585858.0,325.476667,83.84,84.04,94.59,95.95,95.270
4,Griffin High School,Charter,2949,917500.0,311.122414,83.35,83.82,93.39,97.14,95.265
13,Wilson High School,Charter,962,1319574.0,1371.698545,83.27,83.99,93.87,96.54,95.205


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [29]:
Bottom5Schools = dataSummary_byschool.sort_values('% Overall Passing',axis=0,ascending=True)
Bottom5Schools.head(5)

,School Name,Type,Total Students,Total Budgets,Per Student Budgets,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,District,1761,2547363.0,1446.543441,76.84,80.74,66.37,80.22,73.295
2,Figueroa High School,District,4635,1884411.0,406.561165,76.71,81.16,65.99,80.74,73.365
7,Huang High School,District,2283,1910635.0,836.896627,76.63,81.18,65.68,81.32,73.500
8,Johnson High School,District,1858,3094650.0,1665.581270,77.07,80.97,66.06,81.22,73.640
3,Ford High School,District,3999,1763916.0,441.089272,77.10,80.75,68.31,79.30,73.805


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [56]:
# Note: so far, this isn't in grade order
grade_avg_reading_pre = school_data_analysis.groupby(['school_name','grade']).mean()['math_score']
grade_avg_reading_pre
# grade_avg_reading = grade_avg_reading_pre.to_frame().reset_index()
# grade_avg_reading
# type(grade_avg_reading)

school_name            grade
Bailey High School     10th     76.996772
                       11th     77.515588
                       12th     76.492218
                       9th      77.083676
Cabrera High School    10th     83.154506
                       11th     82.765560
                       12th     83.277487
                       9th      83.094697
Figueroa High School   10th     76.539974
                       11th     76.884344
                       12th     77.151369
                       9th      76.403037
Ford High School       10th     77.672316
                       11th     76.918058
                       12th     76.179963
                       9th      77.361345
Griffin High School    10th     84.229064
                       11th     83.842105
                       12th     83.356164
                       9th      82.044010
Hernandez High School  10th     77.337408
                       11th     77.136029
                       12th     77.186567
     

In [67]:
# 9th grade grouping


## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [23]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [24]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.

In [25]:

# # not what I needed here, but cool to have
# # Student totals by School type
# byschool_type = school_data_analysis['type'].value_counts()
# byschool_type
# type(byschool_type)